In [5]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tdmclient.notebook
await tdmclient.notebook.start()

# nouvelle version avec client async

In [21]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node ead09851-d017-4f29-9675-b6d205028bc6

In [ ]:
utiliser time.perf (librairie) pour avoir une référence temporelle pour faire tourner le kalman filter

In [4]:
await tdmclient.notebook.stop()

In [1]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

test_functions = False

In [2]:
await node.wait_for_variables()
#node.var

In [3]:
async def print_sensor_values(sensor_id, print_range=10, delta_time=0.2):
    """
    Print the sensor value sensor_id print_range times, every delta_time seconds
    """
    await node.wait_for_variables({str(sensor_id)})
    for i in range(print_range):
        print(list(node[sensor_id]))
        await client.sleep(delta_time)
        
await print_sensor_values('prox.ground.reflected')

[548, 423]
[548, 421]
[548, 423]
[547, 422]
[549, 422]
[548, 421]
[548, 423]
[548, 422]
[549, 423]
[547, 422]


In [3]:
def motors(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    # Printing the speeds if requested
    if verbose:
        print("\t\t Setting speed : ", l_speed, r_speed)
    return {
        "motor.left.target": [l_speed],
        "motor.right.target": [r_speed],
    }

if test_functions:
    await node.set_variables(motors(100, 100)) #test with lower speed value
    await client.sleep(2)
    await node.set_variables(motors(0, 0))

In [4]:
async def go_straight(motor_speed=100, duration=5, verbose=False):
    """
    goes straight for a certain duration
    """
    if verbose: print("Starting go straight behaviour")
        
    await node.set_variables(motors(motor_speed, motor_speed)) #test with lower speed value
    await client.sleep(duration)
    await node.set_variables(motors(0, 0))
    
if test_functions:
    await go_straight(200,5,True)

In [5]:
def test_see_nothing(close_threshold=1, verbose=False):
    """
    tests if the three middle sensors see nothing
    """
    if any([x>close_threshold for x in node['prox.horizontal'][1:-3]]):
        return False
    if verbose: print("\t\t degagé de l'obstacle")
    return True

In [6]:
async def NN_avoid(motor_speed=100, sensor_scale=400, verbose=False):
    global side
    """
    avoids obstacle with neural network
    """
    see_nothing = False
    y = [0,0] 
    
    while not see_nothing:
        #asymetric w_l[1] != w_r[1] so the robot choses one side if it arrives straight on the obstacle
        w_l = [8, -12, -8, 4, 0]
        w_r = [-8, -8, 8, 0, 4]

        x = [0,0,0,0,0]

        if verbose: print("Starting avoidance with neural network")

        prox_horizontal = node['prox.horizontal'][:-2]
        print(prox_horizontal)
        left_prox = prox_horizontal[0]// sensor_scale
        right_prox = prox_horizontal[4]// sensor_scale

        # Memory
        x[3] = y[0]//10
        x[4] = y[1]//10        

        for i in range(1,4): #we only take the 3 sensors of the middle
            # Get and scale inputs
            x[i-1] = prox_horizontal[i] // sensor_scale
            
        print(x)
        
        y = [80,80] 

        for i in range(len(x)):    
            # Compute outputs of neurons and set motor powers
            y[0] = y[0] + x[i] * w_l[i]
            y[1] = y[1] + x[i] * w_r[i]
        
        print(y)
        
        await client.sleep(0.1)#let the variables be updated
        
        if test_see_nothing(1,True): 
            see_nothing = True
            y = [0,0]
            if left_prox > 1:
                side = 'left'
            if right_prox > 1:
                side = 'right'
                
            print(side)
            
        await node.set_variables(motors(y[0],y[1]))
        

In [7]:
async def turn(motor_speed=100,sensor_scale=400,  verbose=False):
    global side
    """
    turn until obstcale is detected
    """
    adjust = 5
    doneTurn = False
    y = [0,0] 
    
    
    if side == 'none':
        print("side not set")
        doneTurn = True
        
    while not doneTurn:
        
        if verbose: print("turning",side)
            
        prox_horizontal = node['prox.horizontal'][:-2]
        left_prox = prox_horizontal[0]// sensor_scale
        right_prox = prox_horizontal[4]// sensor_scale
        
        print(left_prox)
        print(right_prox)
        
        if side == 'right':
            if right_prox <= 1 + adjust:
                y = [80,0]
            if right_prox > 1 + adjust:
                doneTurn = True
                print("done turn right")
                y = [0,0]
        if side == 'left':
            if left_prox <= 1 + adjust:
                y = [0,80]
            if left_prox > 1 + adjust:
                doneTurn = True
                print("done turn left")
                y = [0,0]
        await client.sleep(0.1)#let the variables be updated
        await node.set_variables(motors(y[0],y[1]))
               

In [8]:
async def follow_obstacle(motor_speed=100,sensor_scale=400, verbose=False):
    global side
    """
    follows the obstacle side until its end
    """
    adjust2 = 2000
    constant_scale = 20
    doneFollow = False
    y = [0,0] 
    
    if side == 'none':
        print("side not set")
        doneFollow = True
        
    while not doneFollow:
        
        if verbose: print("following wall",side)
            
        prox_horizontal = node['prox.horizontal'][:-2]
        left_prox = prox_horizontal[0]// sensor_scale
        right_prox = prox_horizontal[4]// sensor_scale
        
        print(left_prox)
        print(right_prox)
        
        left_prox_adjust = left_prox -adjust2//sensor_scale
        right_prox_adjust = right_prox -adjust2//sensor_scale
        if side == 'right':
            y = [80,80+right_prox_adjust*constant_scale]
        if side == 'left':
            y = [80+left_prox_adjust*constant_scale,80]
        if left_prox < 1 and right_prox < 1:
            doneFollow = True
            y = [0,0]
            
        await client.sleep(0.1)#let the variables be updated
        await node.set_variables(motors(y[0],y[1]))

await follow_obstacle(verbose = True)
               

In [9]:
async def obstacle_avoidance(verbose=True):
    done = False
    side = 'none'
    while not done:
        side = 0;
        await NN_avoid(verbose=verbose)
        
        await go_straight(verbose=verbose)
        
        await turn(verbose=verbose)
        
        await follow_obstacle(verbose=verbose)
        
        await go_straight(verbose=verbose)
        
        done = True
        

In [11]:
await obstacle_avoidance()

# avec onevent

In [6]:
%%run_python #%%transpile_to_aseba # Change to show the code generated in Aseba

#IMPORTANT:
#this programm needs to be launched only when the obsatcle is already detected
#i.e. the sensors already saw something
#otherwise it fucks everything up

# The center button restarts and stops the robot

#sates:
#0 the robot stops 
#1 the robot arrives at the obstacle 
#  and choses which side to go around goes forward until the sensor chose (1 left or 5 right) doesnt see anythin
#2 the robot goes forward approx 1 thymio length
#3 the robot rotates until its sensor (1 or 5) sees the obstacle
#4 the robot follows the obstaclke on its length until its sees nothin
#5 the robot goes forward to completely avoid the obstacle 

adjust2 = 2000
close_thresh = 1
time = 0
state = 1
y = [0,0] 
timer_period[0] = 10
time_one_thym_dist = 150
NONE = 0
side = NONE 
constant_scale = 20
adjust = 5
    
@onevent
def button_center():
    global state
    if button_center == 1:
        state = 1 if state==0 else 0

@onevent
def timer0():
    global prox_horizontal, motor_left_target, motor_right_target, button_center, state, y, NONE, side, time
    #side constants
    LEFT = 1
    RIGHT = 2
    
    #asymetric w_l[1] != w_r[1] so the robot choses one side if it arrives straight on the obstacle
    w_l = [10, -15, -10, 6, 0]
    w_r = [-10, -10, 10, 0, 6]

    # Scale factors for sensors and constant factor
    sensor_scale = 400
    
    x = [0,0,0,0,0]
    
    left_prox = prox_horizontal[0]// sensor_scale
    right_prox = prox_horizontal[4]// sensor_scale
    
### STATE 0 the robot stops###    
    if state == 0:
        time = 0
        y = [0,0]
        
### STATE 1  the robot arrives at the obstacle and choses which side to go around###         
    if state == 1:
        # Memory
        x[3] = y[0]//10
        x[4] = y[1]//10        
        
        for i in range(1,4): #we only take the 3 sensors of the middle
            # Get and scale inputs
            x[i-1] = prox_horizontal[i] // sensor_scale
            
        y = [60,60] 
        tot = x[0]+x[1]+x[2]
        
        for i in range(len(x)):    
            # Compute outputs of neurons and set motor powers
            y[0] = y[0] + x[i] * w_l[i]
            y[1] = y[1] + x[i] * w_r[i]
        
        if tot < close_thresh :#the three middle sensors see nothing
            if left_prox > close_thresh:
                side = LEFT
            if right_prox > close_thresh:
                side = RIGHT
            if left_prox < close_thresh and right_prox < close_thresh: #the thymio turned and stops only if it sees nothin anymore
                state = 2
    
### STATE 2 and 5 : the robot goes forward approx 1 thymio length ###                   
    if state == 2 or state == 5:
        time += 1         
        if time <= time_one_thym_dist:
            y = [160,160]
        elif time > time_one_thym_dist:
            y = [0,0]
            if state == 2:
                time = 0
                state = 3
            if state == 5:
                time = 0
                state = 0
            
### STATE 3 the robot rotates until its sensor (1 or 5) sees the obstacle###   
    if state == 3: 
        if side == RIGHT:
            if right_prox <= close_thresh + adjust:
                y = [160,0]
            if right_prox > close_thresh + adjust:
                state = 4
        if side == LEFT:
            if left_prox <= close_thresh + adjust:
                y = [0,160]
            if left_prox > close_thresh + adjust:
                state = 4
                
### STATE 4 the robot follows the obstacle on its length until its sees nothing###             
    if state == 4:
        left_prox_adjust = left_prox -adjust2//sensor_scale
        right_prox_adjust = right_prox -adjust2//sensor_scale
        if side == RIGHT:
            y = [80,80+right_prox_adjust*constant_scale]
        if side == LEFT:
            y = [80+left_prox_adjust*constant_scale,80]
        if left_prox < close_thresh and right_prox < close_thresh:
            state = 5
        
    # Set motor powers
    motor_left_target = y[0]
    motor_right_target = y[1]
                
            

            